In [ ]:
# === CELL 1: SETUP & IMPORTS ===
!pip install -q yfinance

import yfinance as yf
import pandas as pd
import numpy as np
import json
import time
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ Imports ready")

In [ ]:
# === CELL 2: AGGRESSIVE CONFIG ===
# LET IT TAKE RISKS! Can't learn without making mistakes!

CONFIG = {
    # === ENTRY (AGGRESSIVE!) ===
    'dip_buy_rsi': 50,           # RSI < 50 = opportunity (was 35)
    'dip_buy_drawdown': -2,      # Only need 2% dip (was -8%)
    'bounce_threshold': 2,        # 2% bounce = entry
    'momentum_threshold': 5,      # 5% momentum triggers buy
    
    # === EXIT (LET WINNERS RUN!) ===
    'profit_target_1': 15,
    'profit_target_2': 30,
    'profit_target_max': 50,
    'stop_loss': -15,             # Wide stop
    'trailing_stop': 12,
    'max_hold_days': 60,
    
    # === POSITION SIZING ===
    'max_position_pct': 25,
    'min_cash_reserve': 5,        # Only 5% cash reserve
    'max_positions': 15,          # More positions!
    
    # === TRAINING ===
    'episodes': 2000,
    'initial_balance': 100000,
    'exploration_rate': 0.6,      # 60% exploration!
    'min_exploration': 0.20,      # Keep 20% always
    'buy_boost': 4.0,             # 4x buy probability
}

WATCHLIST = [
    'APLD', 'SERV', 'MRVL', 'NVDA', 'AMD', 'MU', 'QCOM', 'CRDO',
    'SMR', 'OKLO', 'LEU', 'UUUU', 'CCJ',
    'HOOD', 'LUNR', 'SNOW', 'ANET',
    'IONQ', 'RGTI', 'QUBT',
    'TSLA', 'META', 'GOOGL',
    'SPY', 'QQQ',
    'BA', 'RIVN', 'LYFT'
]

print("🔥 AGGRESSIVE CONFIG LOADED!")
print(f"   Watchlist: {len(WATCHLIST)} tickers")
print(f"   Exploration: {CONFIG['exploration_rate']*100:.0f}%")
print(f"   Buy boost: {CONFIG['buy_boost']}x")

In [ ]:
# === CELL 3: LOAD DATA ===
print("📥 Loading market data...")

data_dict = {}
for ticker in WATCHLIST:
    try:
        df = yf.download(ticker, period='2y', progress=False)
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        df = df.reset_index()
        for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        if len(df) > 100:
            data_dict[ticker] = df
            print(f"   ✓ {ticker}: {len(df)} days")
    except Exception as e:
        print(f"   ✗ {ticker}: {e}")

print(f"\n✅ Loaded {len(data_dict)} tickers")

In [ ]:
# === CELL 4: COMPUTE FEATURES ===
def compute_features(df):
    close = df['Close']
    high = df['High']
    low = df['Low']
    volume = df['Volume']
    
    # Returns
    for p in [1, 5, 10, 21]:
        df[f'returns_{p}d'] = close.pct_change(p) * 100
    
    # EMAs
    for p in [8, 13, 21, 34, 55]:
        df[f'ema_{p}'] = close.ewm(span=p).mean()
        df[f'ema_{p}_rising'] = df[f'ema_{p}'] > df[f'ema_{p}'].shift(3)
    
    # Ribbon
    df['ribbon_bullish'] = (df['ema_8'] > df['ema_13']) & (df['ema_13'] > df['ema_21'])
    
    # RSI
    delta = close.diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    df['rsi_14'] = 100 - (100 / (1 + gain / (loss + 1e-10)))
    
    # MACD
    ema12 = close.ewm(span=12).mean()
    ema26 = close.ewm(span=26).mean()
    df['macd'] = ema12 - ema26
    df['macd_signal'] = df['macd'].ewm(span=9).mean()
    df['macd_hist'] = df['macd'] - df['macd_signal']
    df['macd_rising'] = df['macd_hist'] > df['macd_hist'].shift(1)
    
    # Momentum
    df['mom_5d'] = close.pct_change(5) * 100
    
    # Bounce
    df['low_5d'] = low.rolling(5).min()
    df['bounce'] = (close / df['low_5d'] - 1) * 100
    
    # Volume
    df['vol_sma'] = volume.rolling(20).mean()
    df['vol_spike'] = volume > df['vol_sma'] * 1.5
    
    # Trend
    df['trend'] = (np.sign(df['returns_5d']) + np.sign(df['returns_10d']) + np.sign(df['returns_21d'])) / 3
    
    df = df.replace([np.inf, -np.inf], np.nan).ffill().bfill().fillna(0)
    return df

print("🧠 Computing features...")
features = {}
for ticker, df in data_dict.items():
    features[ticker] = compute_features(df.copy())
print(f"✅ Features ready for {len(features)} tickers")

In [ ]:
# === CELL 5: TRADING ENGINE ===
class Trader:
    def __init__(self, balance=100000):
        self.initial = balance
        self.balance = balance
        self.positions = {}
        self.trades = []
    
    def reset(self):
        self.balance = self.initial
        self.positions = {}
        self.trades = []
    
    def value(self, prices):
        v = self.balance
        for t, p in self.positions.items():
            if t in prices:
                v += p['shares'] * prices[t]
        return v
    
    def buy(self, ticker, price, pct=0.20):
        if ticker in self.positions:
            return False
        shares = int(self.balance * pct / price)
        if shares > 0 and shares * price < self.balance * 0.95:
            self.balance -= shares * price
            self.positions[ticker] = {'shares': shares, 'entry': price, 'max': price, 'days': 0}
            return True
        return False
    
    def sell(self, ticker, price):
        if ticker not in self.positions:
            return None
        p = self.positions[ticker]
        pnl = (price / p['entry'] - 1)
        self.balance += p['shares'] * price
        trade = {'ticker': ticker, 'pnl': pnl, 'days': p['days']}
        self.trades.append(trade)
        del self.positions[ticker]
        return trade

print("✅ Trading engine ready")

In [ ]:
# === CELL 6: AGGRESSIVE TRAINING LOOP ===
def run_episode(trader, exploration=0.3):
    trader.reset()
    tickers = list(features.keys())
    min_len = min(len(df) for df in data_dict.values())
    
    for day in range(50, min_len - 1):
        prices = {}
        
        # Update positions
        for t in list(trader.positions.keys()):
            if t in data_dict and day < len(data_dict[t]):
                price = float(data_dict[t]['Close'].iloc[day])
                prices[t] = price
                trader.positions[t]['days'] += 1
                if price > trader.positions[t]['max']:
                    trader.positions[t]['max'] = price
        
        for ticker in tickers:
            if day >= len(features[ticker]):
                continue
            
            df = features[ticker]
            price = float(data_dict[ticker]['Close'].iloc[day])
            prices[ticker] = price
            
            rsi = float(df['rsi_14'].iloc[day])
            mom = float(df['mom_5d'].iloc[day])
            ribbon_bull = bool(df['ribbon_bullish'].iloc[day])
            macd_up = bool(df['macd_rising'].iloc[day])
            ret_21d = float(df['returns_21d'].iloc[day])
            bounce = float(df['bounce'].iloc[day])
            trend = float(df['trend'].iloc[day])
            vol_spike = bool(df['vol_spike'].iloc[day])
            ema8_up = bool(df['ema_8_rising'].iloc[day])
            
            # SELL LOGIC
            if ticker in trader.positions:
                p = trader.positions[ticker]
                pnl = (price / p['entry'] - 1) * 100
                from_max = (price / p['max'] - 1) * 100
                
                sell = False
                if pnl >= CONFIG['profit_target_max']:
                    sell = True
                elif pnl >= CONFIG['profit_target_2'] and rsi > 75:
                    sell = True
                elif pnl >= CONFIG['profit_target_1'] and not ribbon_bull:
                    sell = True
                elif pnl <= CONFIG['stop_loss']:
                    sell = True
                elif pnl > 20 and from_max < -CONFIG['trailing_stop']:
                    sell = True
                elif p['days'] > CONFIG['max_hold_days'] and pnl > 0:
                    sell = True
                
                # Exploration sell
                if np.random.random() < exploration * 0.1:
                    sell = np.random.random() < 0.15
                
                if sell:
                    trader.sell(ticker, price)
            
            # BUY LOGIC (AGGRESSIVE!)
            else:
                if len(trader.positions) >= CONFIG['max_positions']:
                    continue
                if trader.balance / trader.value(prices) < CONFIG['min_cash_reserve'] / 100:
                    continue
                
                signals = 0
                
                # Many ways to trigger a buy!
                if rsi < CONFIG['dip_buy_rsi']:
                    signals += 1
                if ret_21d < CONFIG['dip_buy_drawdown']:
                    signals += 1
                if bounce > CONFIG['bounce_threshold']:
                    signals += 1
                if mom > CONFIG['momentum_threshold']:
                    signals += 1
                if ribbon_bull:
                    signals += 1
                if macd_up:
                    signals += 1
                if trend > 0:
                    signals += 1
                if vol_spike and mom > 0:
                    signals += 1
                if ema8_up:
                    signals += 1
                if rsi < 40 and macd_up:
                    signals += 2
                
                # Buy if ANY signal!
                buy = signals >= 1
                
                # EXPLORATION BUYS (learn from mistakes!)
                if np.random.random() < exploration * CONFIG['buy_boost']:
                    buy = True
                
                if buy:
                    trader.buy(ticker, price, CONFIG['max_position_pct']/100)
    
    # Liquidate
    for t in list(trader.positions.keys()):
        if t in data_dict:
            trader.sell(t, float(data_dict[t]['Close'].iloc[-1]))
    
    return trader.balance, trader.trades

print("✅ Training loop ready")

In [ ]:
# === CELL 7: RUN TRAINING ===
trader = Trader(CONFIG['initial_balance'])

best_return = -999
best_trades = []
results = []

print("=" * 60)
print("🔥 AGGRESSIVE TRAINING - LET IT MAKE MISTAKES!")
print("=" * 60)
print(f"Episodes: {CONFIG['episodes']}")
print(f"Tickers: {len(features)}")
print(f"Exploration: {CONFIG['exploration_rate']*100:.0f}% → {CONFIG['min_exploration']*100:.0f}%")
print("-" * 60)

start = time.time()

for ep in range(CONFIG['episodes']):
    # Exploration decay
    exp = max(CONFIG['min_exploration'],
              CONFIG['exploration_rate'] - ep * (CONFIG['exploration_rate'] - CONFIG['min_exploration']) / CONFIG['episodes'])
    
    final_val, trades = run_episode(trader, exp)
    ret = (final_val / CONFIG['initial_balance'] - 1) * 100
    
    if ret > best_return:
        best_return = ret
        best_trades = trades.copy()
    
    results.append({'ep': ep, 'return': ret, 'trades': len(trades)})
    
    if ep % 100 == 0:
        wins = len([t for t in trades if t['pnl'] > 0])
        wr = wins / max(len(trades), 1) * 100
        print(f"Ep {ep:4d} | Ret: {ret:+6.1f}% | Best: {best_return:+6.1f}% | "
              f"Trades: {len(trades):3d} | WR: {wr:.0f}% | Exp: {exp*100:.0f}%")

print("-" * 60)
print(f"\n🏆 DONE in {time.time()-start:.0f}s")
print(f"   Best Return: {best_return:+.1f}%")
print(f"   Total Trades: {len(best_trades)}")

In [ ]:
# === CELL 8: ANALYZE ===
winners = [t for t in best_trades if t['pnl'] > 0]
losers = [t for t in best_trades if t['pnl'] <= 0]

print("=" * 60)
print("📊 RESULTS")
print("=" * 60)
print(f"Total Trades: {len(best_trades)}")
print(f"Winners: {len(winners)} | Losers: {len(losers)}")
print(f"Win Rate: {len(winners)/max(len(best_trades),1)*100:.1f}%")

if winners:
    print(f"Avg Winner: {np.mean([t['pnl'] for t in winners])*100:+.1f}%")
if losers:
    print(f"Avg Loser: {np.mean([t['pnl'] for t in losers])*100:+.1f}%")

print(f"\n🏆 TOP TRADES:")
for t in sorted(winners, key=lambda x: x['pnl'], reverse=True)[:10]:
    print(f"   {t['ticker']}: {t['pnl']*100:+.1f}% in {t['days']} days")

# By ticker
print(f"\n📈 BY TICKER:")
ticker_pnl = {}
for t in best_trades:
    ticker_pnl.setdefault(t['ticker'], []).append(t['pnl'])

for tk in sorted(ticker_pnl.keys(), key=lambda x: sum(ticker_pnl[x]), reverse=True)[:10]:
    pnls = ticker_pnl[tk]
    print(f"   {tk}: {sum(pnls)*100:+.1f}% | {len(pnls)} trades")

In [ ]:
# === CELL 9: TODAY'S SIGNALS ===
print("=" * 60)
print("🎯 TODAY'S BUY SIGNALS")
print("=" * 60)

signals = []
for ticker in features.keys():
    df = features[ticker]
    price = float(data_dict[ticker]['Close'].iloc[-1])
    rsi = float(df['rsi_14'].iloc[-1])
    mom = float(df['mom_5d'].iloc[-1])
    ribbon_bull = bool(df['ribbon_bullish'].iloc[-1])
    macd_up = bool(df['macd_rising'].iloc[-1])
    ret_21d = float(df['returns_21d'].iloc[-1])
    bounce = float(df['bounce'].iloc[-1])
    trend = float(df['trend'].iloc[-1])
    
    sigs = []
    score = 0
    
    if rsi < 50:
        sigs.append('RSI_LOW')
        score += 1
    if mom > 5:
        sigs.append('MOMENTUM')
        score += 2
    if ribbon_bull:
        sigs.append('RIBBON_BULL')
        score += 2
    if macd_up:
        sigs.append('MACD_UP')
        score += 1
    if bounce > 2:
        sigs.append('BOUNCE')
        score += 1
    if trend > 0:
        sigs.append('TREND_UP')
        score += 1
    if ret_21d < -2:
        sigs.append('DIP')
        score += 1
    
    if sigs:
        signals.append({
            'ticker': ticker,
            'price': price,
            'score': score,
            'signals': sigs,
            'rsi': rsi,
            'mom': mom
        })

signals.sort(key=lambda x: x['score'], reverse=True)

print(f"\n🔥 {len(signals)} STOCKS WITH SIGNALS:\n")
for s in signals:
    stars = '⭐' * min(s['score'], 5)
    sig_str = ' + '.join(s['signals'][:3])
    print(f"{stars} {s['ticker']:6s} ${s['price']:>8.2f} | RSI:{s['rsi']:.0f} Mom:{s['mom']:+.1f}%")
    print(f"         {sig_str}")
    print()